## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

lena = cv2.imread("lena.bmp")

def img_fft(src, testfft: bool = False, show: bool = True, justdecontstuct: bool = False):
    fimage = copy.copy(src)
    if testfft:
        fimage_int = np.fft.fft(fimage, axis=0)
        fimage = np.fft.fft(fimage_int, axis=1)
        fimage = np.fft.fftshift(fimage, [0,1])
        if justdecontstuct and not show:
            return fimage
        fimage = cv2.cartToPolar(fimage.real, fimage.imag)
        fimabs = np.log10(fimage[0] + 1)
        fimarg = fimage[1]
    else:
        fimage = cv2.dft(np.float32(src), flags=cv2.DFT_COMPLEX_OUTPUT)
        fimage = np.fft.fftshift(fimage, [0,1])
        if justdecontstuct and not show:
            return fimage
        fimage = cv2.cartToPolar(fimage[:,:, 0], fimage[:, :, 1])
        fimabs = np.log10(fimage[0] + 1)
        fimarg = fimage[1]
    if show:
        fig, ax = plt.subplots(1, 3)
        fig.set_size_inches(17, 6)
        ax[0].imshow(src)
        ax[0].set_title("src")
        ax[1].imshow(fimabs)
        ax[1].set_title("abs")
        ax[2].imshow(fimarg)
        ax[2].set_title("arg")
    if justdecontstuct:
        return fimage
    return cv2.normalize(fimabs, None, 0, 256, cv2.NORM_MINMAX).astype('uint8'),\
    cv2.normalize(fimarg, None, 0, 256, cv2.NORM_MINMAX).astype('uint8')

def reconstruct(fsrc: cv2, show=True):
    fimg = np.fft.ifftshift(fsrc, [0, 1])
    img = cv2.idft(fimg, flags= cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    img = np.round(cv2.magnitude(img[:, :, 0], img[:, :, 1])).astype('uint8')
    if show:
        fig, ax = plt.subplots()
        fig.set_size_inches(4, 4)
        ax.imshow(img)
    return img

def filter_fft(src: cv2, filterfunction = lambda a: a<=0.1, showfilter: bool = False) -> cv2:
    fimg = img_fft(src, show=False, justdecontstuct=True)
    src_size = src.shape
    FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(src_size[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, src_size[1]]))
    FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(src_size[1]))
    FSpaceColsM = np.outer(np.ones([1, src_size[0]]), FSpaceCols)
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
    FilterF = filterfunction(FreqR).astype(float)
    FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
    filtered_fimg = np.multiply(FilterF3, fimg)

    if showfilter:
        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()
    img = reconstruct(filtered_fimg, show=False)
    return img

In [2]:
def filter_f(src: cv2, hanning_size: int = 21, filterfunction = lambda a: a<=0.1, showfilter: bool = False) -> cv2:
    #fft of src
    fimg = img_fft(src, show=False, justdecontstuct=True)

    #basic filter
    src_size = (hanning_size, hanning_size)
    FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(src_size[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, src_size[1]]))
    FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(src_size[1]))
    FSpaceColsM = np.outer(np.ones([1, src_size[0]]), FSpaceCols)
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
    FilterF = filterfunction(FreqR).astype(float)

    #hanning filter
    hanning = np.hanning(hanning_size)
    hanning2d = np.outer(hanning, hanning)

    #idft of ideal filter
    FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
    FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
    FilterFidft = cv2.idft(np.float32(FilterFRot3),
                    flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

    #make final filter
    Finfilter = np.multiply(FilterFI, hanning2d)
    
    Finfilter = cv2.copyMakeBorder(Finfilter, 0, (src.shape[0]-Finfilter.shape[0]),\
                                0, (src.shape[1]-Finfilter.shape[1]),\
                                     cv2.BORDER_CONSTANT, value= 0)
    Finfilter = cv2.dft(Finfilter, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT | cv2.DFT_COMPLEX_INPUT)
    Finfilter = np.fft.fftshift(Finfilter, [1, 0])

    finfinfilter = np.absolute(Finfilter)

    
    FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
    filtered_fimg = np.multiply(FilterF3, fimg)

    if showfilter:
        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()
    img = reconstruct(filtered_fimg, show=False)
    return img

plt.figure()
plt.imshow(filter_f(lena))